# Old New

In [1]:
    def extract(self, im, rad=None, back=[], fit=False, old=False,
                display=None, plot=None, medfilt=None, nout=None, threads=0):
        """ Extract spectrum given trace(s)
    
            Parameters
            ----------
            im : Data object
                 Input image
            rad : float, default=self.rad
                 Radius for extraction window
            back : array-like of array-like
                 List of two-element lists giving start and end of
                 background window(s), in units of pixels relative to
                 trace location
            nout : integer, default=None
                 Used for multi-object spectra.
                 If not None, specifies number of rows of output image;
                 each extracted spectrum will be loaded into indices
                 loaded into index attribute, with an index for each trace
        """
        if plot is None and display is not None:
            plot = display
    
        hd = dataclass.transpose(im) if self.transpose else copy.deepcopy(im)
    
        if hd.bitmask is None:
            hd.add_bitmask(np.zeros_like(hd.data, dtype=np.uintc))
    
        if fit and (self.sigmodel is None or len(self.sigmodel) == 0):
            raise ValueError('must have a sigmodel to use fit extraction.' +
                             'Use gaussian=True in trace')
    
        if rad is None:
            rad = self.rad
        if back is None:
            back = []
        if len(back) > 0:
            for bk in back:
                if len(bk) != 2 or not isinstance(bk[0], int) or not isinstance(bk[1], int):
                    raise ValueError('back must be list of [backlo, backhi] integer pairs')
    
        nrows = hd.data.shape[0]
        ncols = hd.data.shape[-1]
        spec_shape = (nout if nout is not None else len(self.model), hd.data.shape[1])
    
        spec = np.zeros(spec_shape)
        sig = np.zeros(spec_shape)
        bitmask = np.zeros(spec_shape, dtype=np.uintc)
        background_spectra = np.zeros(spec_shape)  # Abdullah: Added background spectra array
    
        pars = []
        ### Update: Simplified multi-threading handling
        skip, npars = (1, ncols) if threads == 0 else (ncols // threads, threads)
        for col in range(npars):
            ec = ncols if col == threads - 1 else col * skip + skip
            pars.append((hd.data[:, col * skip:ec],
                         hd.uncertainty.array[:, col * skip:ec],
                         hd.bitmask[:, col * skip:ec],
                         np.arange(col * skip, ec),
                         self.model, rad, self.pix0, back, self.sigmodel))
    
        print('  extracting ... ')
    
        if threads > 0:
            ### Update: Used context manager for multiprocessing
            with mp.Pool(threads) as pool:
                if fit:
                    output = pool.map_async(extract_col_fit, pars).get()
                elif old:
                    output = pool.map_async(extract_col_old, pars).get()
                else:
                    output = pool.map_async(extract_col, pars).get()
                col = 0
                for out in output:
                    nc = out[0].shape[1]
                    spec[self.index, col:col + nc] = out[0]
                    sig[self.index, col:col + nc] = out[1]
                    bitmask[self.index, col:col + nc] = out[2]
                    background_spectra[self.index, col:col + nc] = out[3]  # Abdullah: Extracted background spectra
                    col += skip
        else:
            col = 0
            for par in pars:
                if fit:
                    out = extract_col_fit(par)
                elif old:
                    print('  may take some time, consider threads=')
                    out = extract_col_old(par)
                else:
                    out = extract_col(par)
                spec[self.index, col:col + skip] = out[0]
                sig[self.index, col:col + skip] = out[1]
                bitmask[self.index, col:col + skip] = out[2]
                background_spectra[self.index, col:col + skip] = out[3]  # Abdullah: Extracted background spectra
                col += skip
    
        if plot is not None:
            plot.clear()
            plot.tv(hd)
    
        for j, model in enumerate(self.model):
            i = self.index[j]
            if medfilt is not None:
                boxcar = Box1DKernel(medfilt)
                median = convolve(spec[i, :], boxcar, boundary='extend')
                spec[i, :] /= median
                sig[i, :] /= median
    
            if plot is not None:
                cr = model(np.arange(ncols)) + self.pix0
                color = 'b' if i % 2 == 0 else 'm'
                plot.ax.plot(range(ncols), cr, color='g', linewidth=3)
                plot.ax.plot(range(ncols), cr - rad, color=color, linewidth=1)
                plot.ax.plot(range(ncols), cr + rad, color=color, linewidth=1)
                if len(back) > 0:
                    for bk in back:
                        plot.ax.plot(range(ncols), cr + bk[0], color='r', linewidth=1)
                        plot.ax.plot(range(ncols), cr + bk[1], color='r', linewidth=1)
                plot.plotax2.cla()
                plot.plotax2.plot(range(ncols), spec[i], color=color, linewidth=1)
                plot.plotax2.text(0.05, 0.95, 'Extracted spectrum',
                                  transform=plot.plotax2.transAxes)
                plt.draw()
        if plot is not None:
            while getinput('  See extraction window(s). Hit space bar to continue....', plot.fig)[2] != ' ':
                pass
        print("")
        return Data(spec, uncertainty=StdDevUncertainty(sig),
                    bitmask=bitmask, header=hd.header), Data(background_spectra, uncertainty=StdDevUncertainty(sig),
                                                            bitmask=bitmask, header=hd.header)


In [ ]:
def extract_col(pars):
    """ Extract a series of columns, using boxcar extraction for multiple traces """
    data, err, bitmask, cols, models, rad, pix0, back, sigmodels = pars
    spec = np.zeros([len(models), len(cols)])
    sig2 = np.zeros([len(models), len(cols)])
    mask = np.zeros([len(models), len(cols)], dtype=np.uintc)
    background_spec = np.zeros([len(models), len(cols)])  # Abdullah
    ny = data.shape[0]
    ncol = data.shape[1]
    y, x = np.mgrid[0:data.shape[0], 0:data.shape[1]]

    for i, model in enumerate(models):
        # Center of trace
        ymid = model(cols) + pix0

        # Calculate distance of each pixel from trace center
        ylo = max(int(np.min(np.floor(ymid - rad))), 0)
        yhi = min(int(np.max(np.ceil(ymid + rad))), ny - 1)
        dist = y[ylo:yhi + 1, :] - ymid

        # Determine contribution of each pixel to boxcar
        contrib = np.zeros(dist.shape, float)
        # Full pixel contribution
        iy, ix = np.where(np.abs(dist) < rad - 0.5)
        contrib[iy, ix] = 1.0
        # Fractional pixel contribution
        iy, ix = np.where((np.abs(dist) >= rad - 0.5) & (np.abs(dist) <= rad + 0.5))
        contrib[iy, ix] = 1 - (np.abs(dist[iy, ix]) - (rad - 0.5))

        # Add the contributions
        spec[i, :] = np.sum(data[ylo:yhi + 1, :] * contrib, axis=0)
        sig2[i, :] = np.sum(err[ylo:yhi + 1, :]**2 * contrib**2, axis=0)
        # For bitmask, take bitwise_or of pixels that have full contribution
        mask[i, :] = np.bitwise_or.reduce(bitmask[ylo:yhi + 1, :] * contrib.astype(int), axis=0)

        # Background subtraction
        background = np.zeros_like(data[ylo:yhi + 1, :])
        background_err = np.zeros_like(err[ylo:yhi + 1, :])
        if len(back) > 0:
            nback = 0
            for bk in back:
                iy, ix = np.where((dist > bk[0]) & (dist < bk[1]))
                background[iy, ix] = data[iy, ix]
                background_err[iy, ix] = err[iy, ix]**2
                nback += np.abs(bk[1] - bk[0])

            if nback > 0:
                background_spec[i, :] = np.nanmedian(background, axis=0)
                spec[i, :] -= background_spec[i, :] * 2 * rad
                sig2[i, :] += np.nansum(background_err, axis=0) / nback * (2 * rad)

    return spec, np.sqrt(sig2), mask, background_spec


# New New

In [ ]:
    def extract(self, im, rad=None, back=[], fit=False, old=False,
                display=None, plot=None, medfilt=None, nout=None, threads=0):
        """ Extract spectrum given trace(s)
    
            Parameters
            ----------
            hd : Data object
                 Input image
            rad : float, default=self.rad
                 radius for extraction window
            back : array-like of array-like
                 list of two-element lists giving start and end of
                 background window(s), in units of pixels relative to
                 trace location
            nout : integer, default=None
                 used for multi-object spectra.
                 If not None, specifies number of rows of output image;
                 each extracted spectrum will be loaded into indices
                 loaded into index attribute, with an index for each trace
        """
        if plot is None and display is not None:
            plot = display
        if self.transpose:
            hd = dataclass.transpose(im)
        else:
            hd = copy.deepcopy(im)
    
        if hd.bitmask is None:
            hd.add_bitmask(np.zeros_like(hd.data, dtype=np.uintc))
    
        if fit and (self.sigmodel is None or len(self.sigmodel) == 0):
            raise ValueError('must have a sigmodel to use fit extraction.' +
                             'Use gaussian=True in trace')
    
        if rad is None:
            rad = self.rad
        if back is None:
            back = []
        if len(back) > 0:
            for bk in back:
                if len(bk) != 2 or not isinstance(bk[0], int) or not isinstance(bk[1], int):
                    raise ValueError('back must be list of [backlo,backhi] integer pairs')
    
        nrows = hd.data.shape[0]
        ncols = hd.data.shape[-1]
        if nout is not None:
            spec = np.zeros([nout, hd.data.shape[1]])
            sig = np.zeros([nout, hd.data.shape[1]])
            bitmask = np.zeros([nout, hd.data.shape[1]], dtype=np.uintc)
            background_spectra = np.zeros([nout, hd.data.shape[1]])  # Abdullah
        else:
            spec = np.zeros([len(self.model), hd.data.shape[1]])
            sig = np.zeros([len(self.model), hd.data.shape[1]])
            bitmask = np.zeros([len(self.model), hd.data.shape[1]], dtype=np.uintc)
            background_spectra = np.zeros([len(self.model), hd.data.shape[1]])  # Abdullah
    
        pars = []
        if threads == 0:
            pars.append((hd.data, hd.uncertainty.array, hd.bitmask,
                         np.arange(ncols), self.model, rad, self.pix0, back, self.sigmodel))
    
            print('  extracting ... ')
    
            col = 0
            for par in pars:
                if fit:
                    out = extract_col_fit(par)
                elif old:
                    print('  may take some time, consider threads=')
                    out = extract_col_old(par)
                else:
                    out = extract_col(par)
                spec[self.index, col:col + ncols] = out[0]
                sig[self.index, col:col + ncols] = out[1]
                bitmask[self.index, col:col + ncols] = out[2]
                background_spectra[self.index, col:col + ncols] = out[3]  # Abdullah
                col += ncols
        else:
            skip = ncols // threads
            npars = threads
            for col in range(npars):
                if col == threads - 1:
                    ec = ncols
                else:
                    ec = col * skip + skip
                pars.append((hd.data[:, col * skip:ec],
                             hd.uncertainty.array[:, col * skip:ec],
                             hd.bitmask[:, col * skip:ec],
                             np.arange(col * skip, ec),
                             self.model, rad, self.pix0, back, self.sigmodel))
    
            print('  extracting ... ')
    
            pool = mp.Pool(threads)
            if fit:
                output = pool.map_async(extract_col_fit, pars).get()
            elif old:
                output = pool.map_async(extract_col_old, pars).get()
            else:
                output = pool.map_async(extract_col, pars).get()
            pool.close()
            pool.join()
            col = 0
            for out in output:
                nc = out[0].shape[1]
                spec[self.index, col:col + nc] = out[0]
                sig[self.index, col:col + nc] = out[1]
                bitmask[self.index, col:col + nc] = out[2]
                background_spectra[self.index, col:col + nc] = out[3]  # Abdullah back
                col += skip
    
        if plot is not None:
            plot.clear()
            plot.tv(hd)
    
        for j, model in enumerate(self.model):
            i = self.index[j]
            if medfilt is not None:
                boxcar = Box1DKernel(medfilt)
                median = convolve(spec[i, :], boxcar, boundary='extend')
                spec[i, :] /= median
                sig[i, :] /= median
    
            if plot is not None:
                cr = model(np.arange(ncols)) + self.pix0
                color = 'b' if i % 2 == 0 else 'm'
                plot.ax.plot(range(ncols), cr, color='g', linewidth=3)
                plot.ax.plot(range(ncols), cr - rad, color=color, linewidth=1)
                plot.ax.plot(range(ncols), cr + rad, color=color, linewidth=1)
                if len(back) > 0:
                    for bk in back:
                        plot.ax.plot(range(ncols), cr + bk[0], color='r', linewidth=1)
                        plot.ax.plot(range(ncols), cr + bk[1], color='r', linewidth=1)
                plot.plotax2.cla()
                plot.plotax2.plot(range(ncols), spec[i], color=color, linewidth=1)
                plot.plotax2.text(0.05, 0.95, 'Extracted spectrum',
                                  transform=plot.plotax2.transAxes)
                plt.draw()
        if plot is not None:
            while getinput('  See extraction window(s). Hit space bar to continue....', plot.fig)[2] != ' ':
                pass
        print("")
        return Data(spec, uncertainty=StdDevUncertainty(sig),
                    bitmask=bitmask, header=hd.header), Data(background_spectra, uncertainty=StdDevUncertainty(sig),
                                                             bitmask=bitmask, header=hd.header)  # Abdullah

In [ ]:
def extract_col(pars):
    """ Extract a series of columns, using boxcar extraction for multiple traces """
    data, err, bitmask, cols, models, rad, pix0, back, sigmodels = pars
    spec = np.zeros([len(models), len(cols)])
    sig2 = np.zeros([len(models), len(cols)])
    mask = np.zeros([len(models), len(cols)], dtype=np.uintc)
    background_spec = np.zeros([len(models), len(cols)])  # Abdullah
    ny = data.shape[0]
    ncol = data.shape[1]
    y, x = np.mgrid[0:data.shape[0], 0:data.shape[1]]
    pix = np.zeros(data.shape)

    for i, model in enumerate(models):
        # Center of trace
        ymid = model(cols) + pix0

        # Calculate distance of each pixel from trace center
        ylo = int(np.min(np.floor(ymid - rad)))
        yhi = int(np.max(np.ceil(ymid + rad)))
        dist = y[ylo:yhi + 1, :] - ymid

        # Determine contribution of each pixel to boxcar
        contrib = np.zeros(dist.shape, float)
        # Full pixel contribution
        iy, ix = np.where(np.abs(dist) < rad - 0.5)
        contrib[iy, ix] = 1.
        # Fractional pixel contribution
        iy, ix = np.where((np.abs(dist) > rad - 0.5) & (np.abs(dist) < rad + 0.5))
        contrib[iy, ix] = 1 - (np.abs(dist[iy, ix]) - (rad - 0.5))

        # Add the contributions
        spec[i, :] = np.sum(data[ylo:yhi + 1, :] * contrib, axis=0)
        sig2[i, :] = np.sum(err[ylo:yhi + 1, :] ** 2 * contrib ** 2, axis=0)
        # For bitmask take bitwise_or of pixels that have full contribution
        mask[i, :] = np.bitwise_or.reduce(bitmask[ylo:yhi + 1, :] * contrib.astype(int), axis=0)

        # Background
        background = np.empty_like(data)
        background[:] = np.nan
        background_err = np.copy(background)
        if len(back) > 0:
            dist = y - ymid

            nback = 0
            for bk in back:
                iy, ix = np.where((dist > bk[0]) & (dist < bk[1]))
                background[iy, ix] = data[iy, ix]
                background_err[iy, ix] = err[iy, ix] ** 2
                nback += np.abs(bk[1] - bk[0])

            background_spec[i, :] = np.nanmedian(background, axis=0) * 2 * rad  # Abdullah
            spec[i, :] -= np.nanmedian(background, axis=0) * 2 * rad
            sig2[i, :] += np.nansum(background_err, axis=0) / nback * (2 * rad)

    return spec, np.sqrt(sig2), mask, background_spec

In [ ]:
def extract_col(pars):
    """
    Extract a series of columns using boxcar extraction for multiple traces.
    """
    data, err, bitmask, cols, models, rad, pix0, back, sigmodels = pars

    # Initialize arrays for the extracted spectra and background
    num_traces = len(models)
    num_cols = len(cols)
    spec = np.zeros([num_traces, num_cols])
    sig2 = np.zeros([num_traces, num_cols])
    mask = np.zeros([num_traces, num_cols], dtype=np.uintc)
    background_spec = np.zeros([num_traces, num_cols])

    ny, ncol = data.shape
    y, x = np.mgrid[0:ny, 0:ncol]

    for i, model in enumerate(models):
        # Center of trace
        ymid = model(cols) + pix0

        # Calculate the distance of each pixel from the trace center
        ylo = int(np.min(np.floor(ymid - rad)))
        yhi = int(np.max(np.ceil(ymid + rad)))
        dist = y[ylo:yhi+1, :] - ymid

        # Determine contribution of each pixel to boxcar
        contrib = np.zeros(dist.shape, float)
        
        # Full pixel contribution
        full_contrib_indices = np.abs(dist) < (rad - 0.5)
        contrib[full_contrib_indices] = 1.0
        
        # Fractional pixel contribution
        frac_contrib_indices = (np.abs(dist) >= (rad - 0.5)) & (np.abs(dist) < (rad + 0.5))
        contrib[frac_contrib_indices] = 1.0 - (np.abs(dist[frac_contrib_indices]) - (rad - 0.5))

        # Add the contributions
        spec[i, :] = np.sum(data[ylo:yhi+1, :] * contrib, axis=0)
        sig2[i, :] = np.sum(err[ylo:yhi+1, :]**2 * contrib**2, axis=0)

        # For bitmask, take bitwise_or of pixels that have full contribution
        mask[i, :] = np.bitwise_or.reduce(bitmask[ylo:yhi+1, :] * contrib.astype(int), axis=0)

        # Background extraction
        background = np.full_like(data, np.nan)
        background_err = copy.copy(background)
        
        if back:
            dist = y - ymid
            nback = 0
            for bk in back:
                bk_indices = (dist > bk[0]) & (dist < bk[1])
                background[bk_indices] = data[bk_indices]
                background_err[bk_indices] = err[bk_indices]**2
                nback += np.abs(bk[1] - bk[0])

            median_background = np.nanmedian(background, axis=0) * 2 * rad
            background_spec[i, :] = median_background
            spec[i, :] -= median_background
            sig2[i, :] += np.nansum(background_err, axis=0) / nback * (2 * rad)

    return spec, np.sqrt(sig2), mask, background_spec

# New 3 extract_col

In [ ]:
    def extract_col(pars):
        """ Extract a series of columns, using boxcar extraction for multiple traces """
        data, err, bitmask, cols, models, rad, pix0, back, sigmodels = pars
        spec = np.zeros([len(models), len(cols)])
        sig2 = np.zeros([len(models), len(cols)])
        mask = np.zeros([len(models), len(cols)], dtype=np.uintc)
        background_spec = np.zeros([len(models), len(cols)])
        ny, ncol = data.shape
        y, x = np.mgrid[0:ny, 0:ncol]
    
        for i, model in enumerate(models):
            # Center of trace
            ymid = model(cols) + pix0
            ylo = int(np.min(np.floor(ymid - rad)))
            yhi = int(np.max(np.ceil(ymid + rad)))
            dist = y[ylo:yhi + 1, :] - ymid
    
            # Determine contribution of each pixel to boxcar
            contrib = np.zeros(dist.shape, float)
            iy, ix = np.where((np.abs(dist) < rad - 0.5))
            contrib[iy, ix] = 1.0
            iy, ix = np.where((np.abs(dist) >= rad - 0.5) & (np.abs(dist) < rad + 0.5))
            contrib[iy, ix] = 1 - (np.abs(dist[iy, ix]) - (rad - 0.5))
    
            # Add the contributions
            spec[i, :] = np.sum(data[ylo:yhi + 1, :] * contrib, axis=0)
            sig2[i, :] = np.sum(err[ylo:yhi + 1, :] ** 2 * contrib ** 2, axis=0)
            mask[i, :] = np.bitwise_or.reduce(bitmask[ylo:yhi + 1, :] * contrib.astype(int), axis=0)
    
            # Background subtraction
            background = np.full_like(data, np.nan)
            background_err = np.full_like(data, np.nan)
    
            if back:
                dist = y - ymid
                valid_background_pixels = False
    
                for bk in back:
                    if isinstance(bk, list) and len(bk) == 2:
                        b_lo, b_hi = bk
                        iy, ix = np.where((dist > b_lo) & (dist < b_hi))
                        if iy.size > 0:
                            background[iy, ix] = data[iy, ix]
                            background_err[iy, ix] = err[iy, ix] ** 2
                            valid_background_pixels = True
    
                if valid_background_pixels:
                    median_background = np.nanmedian(background, axis=0)
                    background_spec[i, :] = median_background * 2 * rad
                    spec[i, :] -= median_background * 2 * rad
                    nback = np.sum(~np.isnan(background), axis=0)
                    sig2[i, :] += np.nansum(background_err, axis=0) / np.maximum(nback, 1) * (2 * rad)
                else:
                    print("Warning: No valid background pixels found for model index", i)
    
        return spec, np.sqrt(sig2), mask, background_spec

In [ ]:
def extract_col(pars):
    """ Extract a series of columns, using boxcar extraction for multiple traces """
    data, err, bitmask, cols, models, rad, pix0, background_ranges, sigmodels = pars
    spec = np.zeros([len(models), len(cols)])
    sig2 = np.zeros([len(models), len(cols)])
    mask = np.zeros([len(models), len(cols)], dtype=np.uintc)
    background_spec = np.zeros([len(models), len(cols)])
    ny, ncol = data.shape
    y, x = np.mgrid[0:ny, 0:ncol]

    for i, model in enumerate(models):
        # Center of trace
        ymid = model(cols) + pix0
        ylo = int(np.min(np.floor(ymid - rad)))
        yhi = int(np.max(np.ceil(ymid + rad)))
        dist = y[ylo:yhi + 1, :] - ymid

        # Determine contribution of each pixel to boxcar
        contrib = np.zeros(dist.shape, float)
        iy, ix = np.where((np.abs(dist) < rad - 0.5))
        contrib[iy, ix] = 1.0
        iy, ix = np.where((np.abs(dist) >= rad - 0.5) & (np.abs(dist) < rad + 0.5))
        contrib[iy, ix] = 1 - (np.abs(dist[iy, ix]) - (rad - 0.5))

        # Add the contributions
        spec[i, :] = np.sum(data[ylo:yhi + 1, :] * contrib, axis=0)
        sig2[i, :] = np.sum(err[ylo:yhi + 1, :] ** 2 * contrib ** 2, axis=0)
        mask[i, :] = np.bitwise_or.reduce(bitmask[ylo:yhi + 1, :] * contrib.astype(int), axis=0)

        # Background subtraction
        background_collection = []

        for back in background_ranges:
            background = np.full_like(data, np.nan)
            background_err = np.full_like(data, np.nan)

            if back:
                dist = y - ymid
                valid_background_pixels = False

                for bk in back:
                    if isinstance(bk, list) and len(bk) == 2:
                        b_lo, b_hi = bk
                        iy, ix = np.where((dist > b_lo) & (dist < b_hi))
                        if iy.size > 0:
                            background[iy, ix] = data[iy, ix]
                            background_err[iy, ix] = err[iy, ix] ** 2
                            valid_background_pixels = True

                if valid_background_pixels:
                    median_background = np.nanmedian(background, axis=0)
                    background_collection.append((median_background, background_err))
                else:
                    print("Warning: No valid background pixels found for background range", back)

        if background_collection:
            median_backgrounds = np.array([b[0] for b in background_collection])
            average_background = np.nanmedian(median_backgrounds, axis=0)

            background_spec[i, :] = average_background * 2 * rad
            spec[i, :] -= average_background * 2 * rad

            combined_background_err = np.nansum(
                [b[1] for b in background_collection], axis=0) / len(background_collection)
            combined_background_err = np.sqrt(combined_background_err)  # Ensure it's in the correct form
            combined_background_err = np.nanmedian(combined_background_err, axis=0)  # Median along y-axis
            sig2[i, :] += combined_background_err * (2 * rad)

    return spec, np.sqrt(sig2), mask, background_spec

# Good One

In [ ]:
def extract_col(pars):
    """ Extract a series of columns, using boxcar extraction for multiple traces """
    data, err, bitmask, cols, models, rad, pix0, back, sigmodels = pars
    spec = np.zeros([len(models), len(cols)])
    sig2 = np.zeros([len(models), len(cols)])
    mask = np.zeros([len(models), len(cols)], dtype=np.uintc)
    background_spec = np.zeros([len(models), len(cols)])
    ny, ncol = data.shape
    y, x = np.mgrid[0:ny, 0:ncol]

    for i, model in enumerate(models):
        # Center of trace
        ymid = model(cols) + pix0
        ylo = int(np.min(np.floor(ymid - rad)))
        yhi = int(np.max(np.ceil(ymid + rad)))
        dist = y[ylo:yhi + 1, :] - ymid

        # Determine contribution of each pixel to boxcar
        contrib = np.zeros(dist.shape, float)
        iy, ix = np.where((np.abs(dist) < rad - 0.5))
        contrib[iy, ix] = 1.0
        iy, ix = np.where((np.abs(dist) >= rad - 0.5) & (np.abs(dist) < rad + 0.5))
        contrib[iy, ix] = 1 - (np.abs(dist[iy, ix]) - (rad - 0.5))

        # Add the contributions
        spec[i, :] = np.sum(data[ylo:yhi + 1, :] * contrib, axis=0)
        sig2[i, :] = np.sum(err[ylo:yhi + 1, :] ** 2 * contrib ** 2, axis=0)
        mask[i, :] = np.bitwise_or.reduce(bitmask[ylo:yhi + 1, :] * contrib.astype(int), axis=0)

        # Background subtraction
        background_collection = []

        for bac in back:
            background = np.full_like(data, np.nan)
            background_err = np.full_like(data, np.nan)

            if bac:
                dist = y - ymid
                valid_background_pixels = False

                for bk in bac:
                    if isinstance(bk, list) and len(bk) == 2:
                        b_lo, b_hi = bk
                        iy, ix = np.where((dist > b_lo) & (dist < b_hi))
                        if iy.size > 0:
                            background[iy, ix] = data[iy, ix]
                            background_err[iy, ix] = err[iy, ix] ** 2
                            valid_background_pixels = True

                if valid_background_pixels:
                    median_background = np.nanmedian(background, axis=0)
                    background_collection.append((median_background, background_err))
                else:
                    print("Warning: No valid background pixels found for background range", back)

        if background_collection:
            median_backgrounds = np.array([b[0] for b in background_collection])
            average_background = np.nanmedian(median_backgrounds, axis=0)

            background_spec[i, :] = average_background * 2 * rad
            spec[i, :] -= average_background * 2 * rad

            combined_background_err = np.nansum(
                [b[1] for b in background_collection], axis=0) / len(background_collection)
            combined_background_err = np.sqrt(combined_background_err)  # Ensure it's in the correct form
            combined_background_err = np.nanmedian(combined_background_err, axis=0)  # Median along y-axis
            sig2[i, :] += combined_background_err * (2 * rad)

    return spec, np.sqrt(sig2), mask, background_spec